In [1]:
import pandas as pd
import pymysql

def live_db_conn():
    """Live DB 접속 함수"""
    conn = pymysql.connect(
        host = "db-6j3k3.pub-cdb.ntruss.com", 
        user = "redtable",
        password = "fpemxpdlqmf5491!@#",
        autocommit = True, 
        cursorclass = pymysql.cursors.DictCursor, 
        db = "redtable2021")
    return conn


def dev_db_conn():
    """개발 DB 접속 함수"""
    conn = pymysql.connect(
        host = 'db-7ma06.pub-cdb.ntruss.com',
        user = 'redtable',
        password = 'fpemxpdlqmf5491!@#',
        autocommit = True,
        cursorclass = pymysql.cursors.DictCursor)
    return conn

In [2]:
with live_db_conn() as conn:
    sql = """
    SELECT T1.userId, T1.cardNo, T4.name AS 'country', T3.email, T5.id AS 'KONA_CARD_ID'
    FROM redtable2021.KONA_TRANSACTION T1
    LEFT JOIN redtable2021.KTC_USER T2 ON T1.userId = T2.card_user_id AND T2.`type` = 1
    LEFT JOIN redtable2021.user T3 ON T2.user_id = T3.id
    LEFT JOIN redtable2021.IA_KTC_COUNTRY_CODE T4 ON T3.country = T4.code AND T4.lang = 'ko'
    INNER JOIN redtable2021.KONA_CARD T5 ON T1.cardNo = T5.card_no
    WHERE T1.userId != 0
    GROUP BY T1.userId, T1.cardNo
    ORDER BY T1.id ASC
    """
    KONA_USER = pd.read_sql(sql, conn)

with dev_db_conn() as conn:
    sql = """
    SELECT ccTLD, use_nm
    FROM dictionary.dic_domain
    WHERE `status` = 'normal' AND `type` = 'domain' AND ccTLD != '.kr';
    """
    dic_domain = pd.read_sql(sql, conn)
    
    sql = """
    SELECT ccTLD, use_nm
    FROM dictionary.dic_domain
    WHERE `status` = 'normal' AND `type` = 'homepage';
    """
    dic_homepage = pd.read_sql(sql, conn)
    
dic_domain_list = dic_domain["ccTLD"].tolist()
dic_homepage_list = dic_homepage["ccTLD"].tolist()

def country_confirm(email_text, country):
    """이메일을 통해 국적 자동화 함수"""
    try:
        domain = email_text.split("@")[-1]
        domain_detail = "." + email_text.split("@")[-1].split(".")[-1]

        if domain_detail in dic_domain_list:
            return dic_domain[dic_domain["ccTLD"] == domain_detail]["use_nm"].reset_index(drop=True)[0]

        elif domain in dic_homepage_list:
            return dic_homepage[dic_homepage["ccTLD"] == domain]["use_nm"].reset_index(drop=True)[0]

        return country
    except:
        return country

In [3]:
KONA_USER["NLTY"] = KONA_USER.apply(lambda row:(country_confirm(row["email"], row["country"])), axis=1).tolist()

# 수기입력
KONA_USER.loc[KONA_USER["email"] == "soyeon@redtable.kr", "NLTY"] = "대한민국"

In [4]:
KONA_USER

,userId,cardNo,country,email,KONA_CARD_ID,NLTY
0,3101644671,9410411100080021,중국,peternian@gmail.com,8002,중국
1,3101644645,9410411100080112,대한민국,kimduwls3@gmail.com,8011,대한민국
2,3101644832,9410411100080088,None,soyeon@redtable.kr,8008,대한민국
3,3101644687,9410411100080104,대한민국,gimminhye@redtable.kr,8010,대한민국
4,3101644566,9410411100080062,대한민국,ssh@redtable.kr,8006,대한민국
...,...,...,...,...,...,...
1131,3101824763,9410411100056716,중국,690044696@qq.com,5671,중국
1132,3101942496,9410411100055924,중국,280436982@qq.com,5592,중국
1133,3101834707,9410411100181811,중국,2965627210@qq.com,16601,중국
1134,3101946965,9410411100029663,중국,3138905967@qq.com,2966,중국


In [5]:
with live_db_conn() as conn:
    with conn.cursor() as curs:
        sql = """
        UPDATE redtable2021.KONA_SOURCE T1
        SET T1.country = %s
        WHERE T1.KONA_CARD_ID = %s
        """
        val = KONA_USER.apply(lambda row:(row["NLTY"], row["KONA_CARD_ID"]), axis=1).tolist()
        curs.executemany(sql, val)